In [1]:
import numpy as np
import librosa
import glob
import os
from random import randint
import torch
import torch.nn as nn
from torch.utils import data
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import matplotlib.pyplot as plt
%matplotlib inline

import import_ipynb
from ccsn_net import *
from ccsn_dataloader import *

importing Jupyter notebook from ccsn_net.ipynb
importing Jupyter notebook from ccsn_dataloader.ipynb


In [2]:
torch.cuda.set_device(3)
print(torch.cuda.current_device())

3


In [4]:
SENmodel = SEN_classify().double()
state_dict = torch.load('../../model/baseline_ccsn.pkl')
SENmodel.load_state_dict(state_dict)
SENmodel = SENmodel.cuda()
SENmodel = SENmodel.eval()

In [7]:
result_score_dict = {}
result = []
print("len(test_data)",len(my_test_data))

with torch.no_grad():
    for key,token in enumerate(my_test_data):
        x1_name,x2_name,exit_index = load_exits_file(my_test_data,key)
        X1 = torch.from_numpy(np.expand_dims(np.transpose(np.load(x1_name)),axis=2))
        X2 = torch.from_numpy(np.expand_dims(np.transpose(np.load(x2_name)),axis=2))
        X = torch.cat((X1,X2),dim=2)
        X = X.unsqueeze(0)
        X = X.unsqueeze(0) 
        x1 =torch.transpose((X[...,0]),2,3).cuda()
        x2 =torch.transpose((X[...,1]),2,3).cuda()
#         print(x1.shape,x2.shape)
        predict_label = SENmodel(x1,x2).detach().cpu()
        result_score_dict[token] = predict_label[0][0]

print("len(result_score_dict)",len(result_score_dict))


len(test_data) 6000
len(result_score_dict) 6000


In [10]:
from __future__ import division
testLength = 1000

def get_result_key(index,x1,x2):
    return "song"+str(index)+"_"+str(x1)+","+"song"+str(index)+"_"+str(x2)

def get_result_score(index):
    scoreList = []
    score_123 = result_score_dict[get_result_key(index,1,2)] + result_score_dict[get_result_key(index,2,3)]
    score_132 = result_score_dict[get_result_key(index,1,3)] + result_score_dict[get_result_key(index,3,2)]
    score_213 = result_score_dict[get_result_key(index,2,1)] + result_score_dict[get_result_key(index,1,3)]
    score_231 = result_score_dict[get_result_key(index,2,3)] + result_score_dict[get_result_key(index,3,1)]
    score_312 = result_score_dict[get_result_key(index,3,1)] + result_score_dict[get_result_key(index,1,2)]
    score_321 = result_score_dict[get_result_key(index,3,2)] + result_score_dict[get_result_key(index,2,1)]
    scoreList.append(score_123)
    scoreList.append(score_132)
    scoreList.append(score_213)
    scoreList.append(score_231)
    scoreList.append(score_312)
    scoreList.append(score_321)
    import operator
    index, value = max(enumerate(scoreList), key=operator.itemgetter(1))
    return index,value

def cal_accuracy(index):
    if(index==0):
        return 2,True
    elif(index==1):
        return 0,False
    elif(index==2):
        return 0,False
    elif(index==3):
        return 1,False
    elif(index==4):
        return 1,False
    elif(index==5):
        return 0,False
    else:
        return -1,False 

GA = [0,0,0,0,0,0]
PA = 0.0
for i in range(6000,7000):
    index, value = get_result_score(i)
#     print(index,value)
    pair,_ = cal_accuracy(index)
    PA += pair/3
    if(index == 0):
        GA[0] += 1
    if(index == 1):
        GA[1] += 1
    if(index == 2):
        GA[2] += 1
    if(index == 3):
        GA[3] += 1
    if(index == 4):
        GA[4] += 1
    if(index == 5):
        GA[5] += 1

#GA[0] is "123"
#GA[1,2,,3,4,5] is "132,213,231,312,321"
print("Accuracy:",GA[0]/testLength)

# for i in range(6):
#     print("Accuracy:",GA[i]/testLength)

Accuracy: 0.586
